### Step 1: Define the Image Class with Properties

In [72]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import pandas as pd

class ImageData:
    def __init__(self, bending_strength, strength, deformation_strength, deformation_rate, filename, group_name):
        self.bending_strength = bending_strength
        self.strength = strength
        self.deformation_strength = deformation_strength
        self.deformation_rate = deformation_rate
        self.filename = filename
        self.group_name = group_name
        # Define separate paths for top and side views based on filename
        self.top_image_path = f"./Materials_data/{group_name}/{filename}-1.jpg"
        self.side_image_path = f"./Materials_data/{group_name}/{filename}-2.jpg"

    def load_top_image(self):
        # Load the top view image
        image = tf.io.read_file(self.top_image_path)
        image = tf.image.decode_jpeg(image, channels=3)  # Assuming JPEG format as per filenames
        image = tf.image.resize(image, (105, 105))  # Standardize size
        image = image / 255.0  # Normalize to [0, 1]
        return image

    def load_side_image(self):
        # Load the side view image
        image = tf.io.read_file(self.side_image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (105, 105))  # Standardize size
        image = image / 255.0  # Normalize to [0, 1]
        return image
    
# Load the data from Excel
excel_path = './Materials_data/面料库数据表.xlsx'  # Replace with actual file path
data = pd.read_excel(excel_path, sheet_name=None, header=None)

# Initialize a list to store ImageData objects for each image
images = []

# Iterate through the DataFrame in steps of 5 rows per group
for sheet_name, df in data.items():
    for i in range(0, len(df), 5):
        if pd.isna(df.iloc[i, 12]):
            break
        group_name = df.iloc[i, 12]
        bending_strength = df.iloc[i, 1:12].values
        strength = df.iloc[i+1, 1:12].values
        deformation_strength = df.iloc[i+2, 1:12].values
        deformation_rate = df.iloc[i+3, 1:12].values
        filenames = df.iloc[i+4, 1:12].values
        
        # Create ImageData instances
        for j in range(len(filenames)):
            img_data = ImageData(
                bending_strength[j],
                strength[j],
                deformation_strength[j],
                deformation_rate[j],
                filenames[j],
                group_name
            )
            images.append(img_data)

# Display processed data for verification
for idx, img_data in enumerate(images[:5], start=1):  # Display first 5 entries for verification
    print(f"Image {idx}:")
    print(f"  Group: {img_data.group_name}")
    print(f"  Bending Strength: {img_data.bending_strength}")
    print(f"  Strength: {img_data.strength}")
    print(f"  Deformation Strength: {img_data.deformation_strength}")
    print(f"  Deformation Rate: {img_data.deformation_rate}")
    print(f"  Filename: {img_data.filename}")
    print(f"  Top Image Path: {img_data.top_image_path}")
    print(f"  Side Image Path: {img_data.side_image_path}")
    print("\n")


Image 1:
  Group: 第一组
  Bending Strength: 0
  Strength: 20
  Deformation Strength: 20
  Deformation Rate: 20
  Filename: 0.20.20.20
  Top Image Path: ./Materials_data/第一组/0.20.20.20-1.jpg
  Side Image Path: ./Materials_data/第一组/0.20.20.20-2.jpg


Image 2:
  Group: 第一组
  Bending Strength: 10
  Strength: 20
  Deformation Strength: 20
  Deformation Rate: 20
  Filename: 10.20.20.20
  Top Image Path: ./Materials_data/第一组/10.20.20.20-1.jpg
  Side Image Path: ./Materials_data/第一组/10.20.20.20-2.jpg


Image 3:
  Group: 第一组
  Bending Strength: 20
  Strength: 20
  Deformation Strength: 20
  Deformation Rate: 20
  Filename: 20.20.20.20
  Top Image Path: ./Materials_data/第一组/20.20.20.20-1.jpg
  Side Image Path: ./Materials_data/第一组/20.20.20.20-2.jpg


Image 4:
  Group: 第一组
  Bending Strength: 30
  Strength: 20
  Deformation Strength: 20
  Deformation Rate: 20
  Filename: 30.20.20.20
  Top Image Path: ./Materials_data/第一组/30.20.20.20-1.jpg
  Side Image Path: ./Materials_data/第一组/30.20.20.20-2.jpg




### Step 2: Create Pairs for Siamese Network

In [73]:
import random

def create_pairs(images):
    image_pairs = []
    labels = []
    # Pair images within the same group (similar) and across groups (dissimilar)
    for i in range(len(images)):
        for j in range(i+1, len(images)):
            if images[i].group_name == images[j].group_name:
                label = 1  # Similar
            else:
                label = 0  # Dissimilar
            image_pairs.append((images[i], images[j]))
            labels.append(label)
    return image_pairs, labels

image_pairs, labels = create_pairs(images)


### Step 3: Preprocess and Load Image Data

In [75]:
def preprocess_image_pair(image_pair):
    img1 = image_pair[0].load_top_image()  # Load the top view for the first image
    img2 = image_pair[1].load_side_image()  # Load the side view for the second image
    return img1, img2

# Process all image pairs
left_images = []
right_images = []

for pair in image_pairs:
    img1, img2 = preprocess_image_pair(pair)
    left_images.append(img1)
    right_images.append(img2)

# Convert lists to numpy arrays for model input
left_images = np.array(left_images)
right_images = np.array(right_images)
labels = np.array(labels)


2024-11-10 00:08:39.526199: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: ./Materials_data/第七组/30.40.40.20-2.jpg; No such file or directory


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} ./Materials_data/第七组/30.40.40.20-2.jpg; No such file or directory [Op:ReadFile]

### Step 4: Define and Compile the Siamese Network Model

In [ ]:
def build_base_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (10, 10), activation='relu', input_shape=(105, 105, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (7, 7), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (4, 4), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, (4, 4), activation='relu'),
        layers.Flatten(),
        layers.Dense(4096, activation='sigmoid')
    ])
    return model

def build_siamese_model():
    input_left = layers.Input(name="left_input", shape=(105, 105, 3))
    input_right = layers.Input(name="right_input", shape=(105, 105, 3))

    base_model = build_base_model()

    output_left = base_model(input_left)
    output_right = base_model(input_right)

    # Compute L1 distance between features
    l1_distance = tf.abs(output_left - output_right)
    output = layers.Dense(1, activation='sigmoid')(l1_distance)

    siamese_model = Model(inputs=[input_left, input_right], outputs=output)
    return siamese_model

# Compile the model
siamese_model = build_siamese_model()
siamese_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


### Step 5: Train the model

In [4]:
# Train the model
siamese_model.fit([left_images, right_images], labels, batch_size=32, epochs=10)


### Step 6: Make Predictions

In [5]:
def predict_similarity(img_data1, img_data2):
    img1 = img_data1.load_image()
    img2 = img_data2.load_image()
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    return siamese_model.predict([img1, img2])[0][0]

# Example usage
similarity_score = predict_similarity(images[0], images[1])
print("Similarity score:", similarity_score)
